In [53]:
from sympy import symbols, Function, Eq, dsolve, Derivative 

t, k, m, A = symbols('t k m A', real=True)

x1, x2 = symbols("x_1 x_2", cls=Function, real=True)

eqns = [Eq(Derivative(x1(t),t,2), -2*x1(t)+x2(t)), Eq(Derivative(x2(t),t,2), x1(t)-2*x2(t))]
# display(eqns[0])
# display(eqns[1])

sol = dsolve(eqns, [x1(t), x2(t)])
# display(sol[0])
# display(sol[1])

solution = dsolve(eqns, [x1(t), x2(t)], ics={Derivative(x1(t),t,1).subs(t,0): 0, Derivative(x2(t),t,1).subs(t,0):0, x1(0): 1, x2(0): 0})
display(solution[0])
display(solution[1])

Eq(x_1(t), cos(t)/2 + cos(sqrt(3)*t)/2)

Eq(x_2(t), cos(t)/2 - cos(sqrt(3)*t)/2)

In [101]:
from sympy import symbols, Function, Eq, dsolve, solve, Derivative, Matrix, MatrixSymbol, MatMul

t, k, m, A, x_vec = symbols('t k m A  x_vec', real=True)

x1, x2 = symbols("x_1 x_2", real=True)

x_vec = symbols('t k m A  x_vec', real=True, cls=Function)

matrix = Matrix([[-2, 1], 
                 [1, -2]])
A= (k/m) * matrix
x_vec = Matrix([x1, x2])
# X = MatrixSymbol('X', 2, 1)

RHS = (A * x_vec)
LHS=Matrix([Derivative(x1, t, 2), Derivative(x2,t,2)])
display(LHS)
display(RHS)

eqn=Eq(LHS,RHS)
# display(LHS-RHS)
eqn = (LHS-RHS).T.tolist()[0]

solution = solve(eqn, x1, x2)
display(solution)
display(solution.tolist())
# display(solution[0])
type(solution.values())

Matrix([
[Derivative(x_1, (t, 2))],
[Derivative(x_2, (t, 2))]])

Matrix([
[-2*k*x_1/m + k*x_2/m],
[ k*x_1/m - 2*k*x_2/m]])

{x_1: (-2*m*Derivative(x_1, (t, 2)) - m*Derivative(x_2, (t, 2)))/(3*k),
 x_2: (-m*Derivative(x_1, (t, 2)) - 2*m*Derivative(x_2, (t, 2)))/(3*k)}

AttributeError: 'dict' object has no attribute 'tolist'

In [9]:
from sympy import MatrixSymbol, Matrix
X = MatrixSymbol('X', 3, 3)
display(X)

X